In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

import pandas as pd
import xlwings as xw
import time

In [5]:
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument('disable-notifications')

# driver = webdriver.Chrome(ChromeDriverManager().install())
chrome_driver_path = "chromedriver.exe"
driver = webdriver.Chrome(executable_path=chrome_driver_path, options = chrome_options)
url = 'https://www.eccmid.org/sponsorship-and-exhibition/sponsor-list'
driver.get(url)
driver.maximize_window()
time.sleep(1)
accept_xpath = '/html/body/div[1]/div[2]/span[1]'
accept = driver.find_element(By.XPATH, accept_xpath)
accept.click()
time.sleep(1)
close_xpath = '//*[@id="close"]/img'
close = driver.find_element(By.XPATH, close_xpath)
close.click()

In [6]:
# Define a function to scroll to the bottom of the page
def scroll_to_bottom():
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# Get the initial page height
initial_height = driver.execute_script("return document.body.scrollHeight")
# Perform initial scroll
scroll_to_bottom()
# Wait for some time to let the content load
time.sleep(2)
# Keep scrolling until the page height stops increasing
while True:
    # Scroll to the bottom of the page
    scroll_to_bottom()
    # Wait for some time to let the content load
    time.sleep(2)
    # Get the updated page height
    new_height = driver.execute_script("return document.body.scrollHeight")
    # Check if the page height has stopped increasing
    if new_height == initial_height:
        break
    # Update the initial height for the next iteration
    initial_height = new_height

In [12]:
div_xpath = '//*[@id="c41055"]/div/div[2]'

# Find the div element
div_element = driver.find_element(By.XPATH, div_xpath)

# Find all the div elements within the div
text_elements = div_element.find_elements(By.XPATH, ".//div")

# Find all the a elements within the div (links)
a_elements = div_element.find_elements(By.XPATH, ".//a")

# Extract text and links
text_list = [a.text for a in text_elements]
link_list = [a.get_attribute("href") for a in a_elements]

In [18]:
text_list = [text for text in text_list if text != '']

In [20]:
# extract austeller und booth
name = []
booth = []
for i in range(len(text_list)):
    name.append(text_list[i].split('\n')[0])
    booth.append(text_list[i].split('\n')[-1].split('Stand')[-1].strip())

In [23]:
# Excel ausfüllen
wb = xw.Book('2023 AACC Ausstellerliste_Vorlage.xlsx')
sheet = wb.sheets[0]

A_col_range = sheet.range('A3:A{}'.format(3+len(name)))
A_col_range.value = [[value] for value in name]

I_col_range = sheet.range('I3:I{}'.format(3+len(booth)))
I_col_range.value = [[value] for value in booth]

In [24]:
# add hyperlinks
for i in range(len(link_list)):
    row = 3+i    
    cell_row, cell_column = row, 1
    cell_address = (cell_row, cell_column)
    sheet.range(cell_address).add_hyperlink(link_list[i], text_to_display=name[i])
    sheet.range(cell_address).api.Font.Bold = True
    sheet.range(cell_address).font.color = (0, 0, 0)
    sheet.range(cell_address).api.Font.Size = 14
    sheet.range(cell_address).api.Font.Name = 'Verdana'